In [1]:
import dotenv
dotenv.load_dotenv()

True

# Introduction to Chains

In this notebook we will go through the most popular usecase for LangChain which is building Chains.

### What is a Chain?

Chain consists of chardcoded list of steps, which one can later execute on custom input. 
Recap what we did in the previous notebook, first we formatted the prompt and then we send it to the model for processing.
This is the simples example of a chain. Later we will explore more complex ones.


### LangChain Expression Language (LCEL)

LCEL is a declarative way to easily write and compose custom chains.
Apart from enabling developers to do it in a very pythonic way, it has several other benefits.
Any Chain constructed this way will automatically have full sync, async, batch, and streaming support. Also any components that can be run in parallel automatically are.

Let's start with rewriting the chain from previous notebook:

In [2]:
### imports
from langchain.chat_models import ChatOpenAI 
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.schema.messages import HumanMessage
from langchain.schema.output_parser import StrOutputParser

In [4]:
behaviour = "an expert at telling jokes"
topic = "bear"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are {behaviour}."),
        HumanMessage(content="Hi. What's your name?"),
        AIMessagePromptTemplate.from_template("Hello. I am {behaviour}. What can I do for you today?"),
        HumanMessagePromptTemplate.from_template("Tell me a joke about {topic}")
    ]
)

chat_api = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0)

# instead of
# chat_resp = chat_api(chat_prompt.format_messages(behaviour=behaviour, topic=topic))
# we can write
chat_chain = chat_prompt | chat_api | StrOutputParser()

res = chat_chain.invoke({"behaviour": behaviour, "topic": topic})
print(res)

Sure, here's a bear-themed joke for you:

Why don't bears wear shoes?

Because they have bear feet!


In [5]:
chat_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'behaviour': {'title': 'Behaviour', 'type': 'string'},
  'topic': {'title': 'Topic', 'type': 'string'}}}

In [6]:
chat_chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

### Building blocks

As you can see, the code is cleaner and easy to understand.

Now let's dig deeper and understand how those building blocks can be chained together.

It is important to remember, that the function used in chain must accept only one argument. 
So if you have a function which takes more than one argument, you can refactor it, to take a dictionary of inputs as argument or call it from another function which does so.

In [28]:
from langchain.schema.runnable import (
    RunnableLambda, 
    RunnableMap, 
    RunnableBranch,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

`RunnableLambda` is a wrapper around arbitrary function which makes it compatible with Expression Language.

In [10]:
chain = RunnableLambda(lambda x: x + 1) | (lambda x: x * 7) | (lambda x: x - 1) # type: ignore
chain.invoke(1)

13

In [11]:
def my_custom_fn(a, b, c, d):
    return f"{a} {b} {c} {d}"

def fn_wrapper(inputs):
    a = inputs["a"]
    b = inputs["b"]
    c = inputs["c"]
    d = inputs["d"]
    return my_custom_fn(a, b, c, d)


chain = RunnableLambda(fn_wrapper)
chain.invoke({"a": 1, "b": 2, "c": 3, "d": 4})

'1 2 3 4'

In [12]:
# or using the inspect module
from inspect import signature
def fn_wrapper2(inputs):
    fn_sig = signature(my_custom_fn)
    return my_custom_fn(**{k: inputs[k] for k in fn_sig.parameters.keys()})

chain = RunnableLambda(fn_wrapper2)
chain.invoke({"a": 1, "b": 2, "c": 3, "d": 4})

'1 2 3 4'

`RunnableMap` can execute multiple Runnables in parallel and return the output of these Runnables as a map.

Each element of the map will get the same input and all the outputs will be combined into single dictionary.

Note that the input is passed as in standard Python function call, so it can be modified by the function!

In [27]:
def f1(inputs):
    print(f"f1 inputs: {inputs}\n")
    inputs.append('f1')
    return inputs

def f2(inputs):
    print(f"f2 inputs: {inputs}\n")
    inputs_copy = inputs.copy()
    inputs_copy.append('f2')
    return inputs_copy

def f3(inputs):
    print(f"f3 inputs: {inputs}\n")
    return {"some_name": "f3"}

chain = RunnableMap({
    "f1 return value": f1,
    "f2 return value": f2,
    "f3 return value": f3
})
chain.invoke([1, 2, 3])

f1 inputs: [1, 2, 3]
f2 inputs: [1, 2, 3]


f3 inputs: [1, 2, 3, 'f1']



{'f1 return value': [1, 2, 3, 'f1'],
 'f2 return value': [1, 2, 3, 'f2'],
 'f3 return value': {'some_name': 'f3'}}

You can also use `itemgetter` to extract arguments from input dictionary.

In [30]:
from operator import itemgetter

def f1(x):
    return x + 1

def f2(y):
    return y + " world"

chain = RunnableMap({
    "f1": itemgetter("x") | RunnableLambda(f1),
    "f2": itemgetter("y") | RunnableLambda(f2),
})

chain.invoke({"x": 1, "y": "hello"})


{'f1': 2, 'f2': 'hello world'}

`RunnableMap` runs in parallel

In [31]:
import time
from operator import itemgetter

sleep_chain = RunnableMap({
    't1': itemgetter('t1') | RunnableLambda(time.sleep), 
    't2': itemgetter('t2') | RunnableLambda(time.sleep),
    't3': itemgetter('t3') | RunnableLambda(time.sleep),
})

start = time.time()
sleep_chain.invoke({'t1': 3, 't2': 3, 't3': 3})
end = time.time()   
print(end - start)

3.0062379837036133


#### It is better to use explicit `Runnable` types when chaining with `|` operator

You can chain runnable map with Python dictionary  using pipe `|` operator, but be carefull to do it on one dictionary, not few in a row.

For example, this chain will not work as you might expect, insted the pipe operator will be interpreted as dict union operator:

In [32]:
# type: ignore
good_chain = RunnableMap({
    'x': lambda inputs: inputs['x'] + 'first ',
}) | RunnableMap({
    'x': lambda inputs: inputs['x'] + 'second ',
}) | RunnableMap({
    'x': lambda inputs: inputs['x'] + 'last',
}) 

bad_chain = {
    'x': lambda inputs: inputs['x'] + 'first ',
} | {
    'x': lambda inputs: inputs['x'] + 'second ',
} | RunnableMap({
    'x': lambda inputs: inputs['x'] + 'last',
}) 

good_chain.invoke({'x': 'start '}), bad_chain.invoke({'x': 'start '})

({'x': 'start first second last'}, {'x': 'start second last'})

In [155]:
good_chain

{
  x: RunnableLambda(...)
}
| {
    x: RunnableLambda(...)
  }
| {
    x: RunnableLambda(...)
  }

In [156]:
bad_chain

{
  x: RunnableLambda(...)
}
| {
    x: RunnableLambda(...)
  }

`RunnablePassthrough` passes the whole input further.

It can be used to wrap a input into dictionary to provide keys necessary for some function, or make the input available in later chain steps.

In [88]:
def search(query):
    return "search results"

def combine_results(results):
    query = results["query"]
    search_results = results["search_results"]

    return f"results for '{query}': {search_results}"

chain = RunnableMap({
    "query": RunnablePassthrough(),
    "search_results": RunnableLambda(search),
}) | RunnableLambda(combine_results)

chain.invoke("I am looking for ...")

"results for 'I am looking for ...': search results"

`RunnableBranch` is used for conditional branching.

In [38]:
def handle_int(x):
    print("handling int")
    return x + 1

def handle_str(x):
    print("handling str")
    return x + " world"

def default_handler(x):
    print("handling default")
    return x

chain = RunnableBranch(
    (lambda x: isinstance(x, int), RunnableLambda(handle_int)),
    (lambda x: isinstance(x, str), RunnableLambda(handle_str)),
    default_handler
)

# or passing a function that handles routing
def route(inputs):
    if isinstance(inputs, int):
        return handle_int(inputs)
    elif isinstance(inputs, str):
        return handle_str(inputs)
    else:
        return default_handler(inputs)
    
chain2 = RunnableLambda(route)

chain.invoke(1), chain.invoke("hello"), chain2.invoke(1.0)

handling int
handling str
handling default


(2, 'hello world', 1.0)

### Storing chat history with `ConversationBufferMemory`

If `return_messages=True` it returns a list of messages in Human/AI format, otherwise it returns a string with predefined prefixes.
Only one key per inputs and outputs is allowed by now.

In [46]:
# type: ignore
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        'question': 'What is 2+2?',
        # 'question2': 'What is 3+3?' # try uncommenting this line to see what happens
    },
    outputs={
        'answer': '4'
    }
)
memory.load_memory_variables(None)

{'history': [HumanMessage(content='What is 2+2?'), AIMessage(content='4')]}

In [47]:
memory = ConversationBufferMemory(return_messages=False, memory_key='my_custom_key', ai_prefix='BOT')

memory.save_context(
    inputs={
        'question1': 'What is 2+2?'
    },
    outputs={
        'answer': '4'
    }
)
memory.load_memory_variables({})

{'my_custom_key': 'Human: What is 2+2?\nBOT: 4\nHuman: What is 2+2?\nBOT: 4'}

### Exercise: 
##### Create a chain for chatting with the model and storing conversation history.

There is already a `ConversationChain` implemented in LangChain but it puts everything into one Human prompt message and doesn't take advantage of System/Human/AI messages.

You task is to create a chain which will use them.

In [50]:
# type: ignore
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0),
    verbose=True,
    memory=ConversationBufferMemory(return_messages=False)
)

conversation.invoke("What is 2+2?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 2+2?
AI:

> Finished chain.


{'input': 'What is 2+2?', 'history': '', 'response': '2+2 is equal to 4.'}

In [51]:
conversation.invoke("What was the result? I forgot.") # type: ignore



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 2+2?
AI: 2+2 is equal to 4.
Human: What was the result? I forgot.
AI:

> Finished chain.


{'input': 'What was the result? I forgot.',
 'history': 'Human: What is 2+2?\nAI: 2+2 is equal to 4.',
 'response': 'The result of 2+2 is 4.'}

In [ ]:
#type: ignore
from typing import Dict
from langchain.memory import ConversationBufferMemory
from langchain.schema.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder # Prompt template that assumes variable is already list of messages.


prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful chatbot."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

def get_question_and_history(input_string: str) -> Dict[str, str]:

    return {
        "question": ...,
        "history": memory... # get list of history messages from memory
    }

def get_answer_and_pass_question_through(input_dict: Dict[str, str]) -> Dict[str, str]:

    get_answer_chain = ... # format prompt > call model > extract output from AI Message object

    return {
        "answer": get_answer_chain.invoke(...), # recall from earlier, that prompt accepts a dictionary with key names == template variables
        "question": ..., # pass question further 
    }

def update_history_and_return_answer(inputs: Dict[str, str]) -> str:
    # we use question/answer keys because they are required by the object, but it doesn't matter
    memory.save_context({'question': ...}, {'answer': ...}) # store both question and answer in memory object, 
    
    return ... # return answer string

chat_chain = RunnableLambda(get_question_and_history) | RunnableLambda(get_answer_and_pass_question_through) | RunnableLambda(update_history_and_return_answer) | StrOutputParser()


print(chat_chain.invoke("Hi. What's your name?"))
print(chat_chain.invoke("What is 2+2?"))

# see list of history messages
print(...)

### Solution

In [54]:
from typing import Dict
from langchain.memory import ConversationBufferMemory
from langchain.schema.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful chatbot."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


def get_question_and_history(input_str: str) -> Dict[str, str]:
    return {
        "question": input_str,
        "history": memory.load_memory_variables(inputs=None)['history'], # type: ignore
    }

def get_answer_and_pass_question_through(input_dict: Dict[str, str]) -> Dict[str, str]:
    get_answer_chain = prompt | model | StrOutputParser()
    return {
        "question": input_dict['question'],
        "answer": get_answer_chain.invoke(input_dict),
    }

def update_history_and_return_answer(inputs: Dict[str, str]) -> str:
    memory.save_context({'question': inputs['question']}, {'answer': inputs['answer']})

    return inputs['answer']

chat_chain = RunnableLambda(get_question_and_history) | RunnableLambda(get_answer_and_pass_question_through) | RunnableLambda(update_history_and_return_answer)

print(chat_chain.invoke("Hi. What's your name?"))
print(chat_chain.invoke("What is 2+2?"))

print(memory.load_memory_variables({})['history'])

Hello! I am a chatbot, so I don't have a personal name. You can simply refer to me as "chatbot" or "assistant." How can I assist you today?
The sum of 2 and 2 is 4.
[HumanMessage(content="Hi. What's your name?"), AIMessage(content='Hello! I am a chatbot, so I don\'t have a personal name. You can simply refer to me as "chatbot" or "assistant." How can I assist you today?'), HumanMessage(content='What is 2+2?'), AIMessage(content='The sum of 2 and 2 is 4.')]
